# Homework 2

In this project on sequence-to-sequence models using PyTorch and TorchText, we'll be implementing the model from [Learning Phrase Representations using RNN Encoder-Decoder for Statistical Machine Translation](https://arxiv.org/abs/1406.1078)

## Introduction

Let's remind ourselves of the general encoder-decoder model.

![](assets/seq2seq1.png)

We use our encoder (green) over the source sequence to create a context vector (red). We then use that context vector with the decoder (blue) and a linear layer (purple) to generate the target sentence.
We'll also be using a GRU (Gated Recurrent Unit) instead of an LSTM (Long Short-Term Memory).

## Preparing Data



We'll import PyTorch, TorchText, spaCy and a few standard modules for preparing the data

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np

from torchtext.legacy.datasets import TranslationDataset, Multi30k
from torchtext.legacy.data import Field, BucketIterator

import spacy

import random
import math
import time


Then set a random seed for deterministic results/reproducability.

In [2]:
SEED = 1234

random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

Run the cell below only if you are using google collab. It will download the german spacy models to google collab

In [3]:
# COLLAB = False
COLLAB = True

if COLLAB:
    import spacy.cli
    spacy.cli.download("de")
    spacy.cli.download("de_core_news_sm")


⚠ As of spaCy v3.0, shortcuts like 'de' are deprecated. Please use the full
pipeline package name 'de_core_news_sm' instead.
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')


Instantiate our German and English spaCy models.

In [4]:
spacy_de = spacy.load("de_core_news_sm")
spacy_en = spacy.load('en_core_web_sm')


ModuleNotFoundError: No module named 'absl'

In [ ]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings
    """
    return [tok.text for tok in spacy_de.tokenizer(text)]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

Create our fields to process our data. This will append the "start of sentence" and "end of sentence" tokens as well as converting all words to lowercase.

In [ ]:
SRC = Field(tokenize=tokenize_de, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

TRG = Field(tokenize = tokenize_en, 
            init_token='<sos>', 
            eos_token='<eos>', 
            lower=True)

Load our data.

In [ ]:
train_data, valid_data, test_data = Multi30k.splits(exts = ('.de', '.en'), 
                                                    fields = (SRC, TRG))

We'll also print out an example just to double check everything is ok.

In [ ]:
print(vars(train_data.examples[0]))

{'src': ['zwei', 'junge', 'weiße', 'männer', 'sind', 'im', 'freien', 'in', 'der', 'nähe', 'vieler', 'büsche', '.'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}


Then create our vocabulary, converting all tokens appearing less than twice into `<unk>` tokens.

In [ ]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

Finally, define the `device` and create our iterators.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [ ]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

## Building the Seq2Seq Model

### Encoder

The encoder is a single-layer GRU. We also don't pass the dropout as an argument to the GRU as that dropout is used between each layer of a multi-layered RNN. As we only have a single layer, PyTorch will display a warning if we try and use pass a dropout value to it.

Another thing to note about the GRU is that it only requires and returns a hidden state, there is no cell state like in the LSTM.

$$\begin{align*}
h_t &= \text{GRU}(x_t, h_{t-1})\\
(h_t, c_t) &= \text{LSTM}(x_t, (h_{t-1}, c_{t-1}))\\
h_t &= \text{RNN}(x_t, h_{t-1})
\end{align*}$$

Inside the GRU, however, is a number of *gating mechanisms* that control the information flow in to and out of the hidden state (similar to an LSTM). 

The encoder takes in a sequence, $X = \{x_1, x_2, ... , x_T\}$, recurrently calculates hidden states, $H = \{h_1, h_2, ..., h_T\}$, and returns a context vector (the final hidden state), $z=h_T$.

$$h_t = \text{EncoderGRU}(x_t, h_{t-1})$$



![](assets/seq2seq5.png)

In [ ]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, dropout):
        super().__init__()
        
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.dropout = dropout  # do we need this?
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        self.rnn = nn.GRU(emb_dim, hid_dim)
        self.dropout = nn.Dropout(p=dropout, inplace=True)
        
    def forward(self, src): # src.shape = [23, 128]
        # The dropout layer randomly zeroes some of the elements of the embedding with probability p using samples from a Bernoulli distribution.
        embedded = self.dropout(self.embedding(src)) # [23, 128, 256]
        unused_output, hidden = self.rnn(embedded) 
        return hidden  # [1, 128, 512]


## Decoder


The GRU in the decoder takes 3 inputs the target token, $y_t$ the previous hidden state $s_{t-1}$ and the context vector $z$. 

$$s_t = \text{DecoderGRU}(y_t, s_{t-1}, z)$$

We re-use the same context vector returned by the encoder for every time-step in the decoder. 

 We pass the current token, $\hat{y}_t$ and the context vector, $z$ to the linear layer.

$$\hat{y}_{t+1} = f(y_t, s_t, z)$$

Thus, our decoder now looks something like this:

![](assets/seq2seq6.png)

Note, the initial hidden state, $s_0$, is still the context vector, $z$, so when generating the first token we are actually inputting two identical context vectors into the GRU.



Within the implementation, we will pass $y_t$ and $z$ to the GRU by concatenating them together, so the input dimensions to the GRU are now `emb_dim + hid_dim` (as context vector will be of size `hid_dim`). The linear layer will take $y_t, s_t$ and $z$ also by concatenating them together, hence the input dimensions are now `emb_dim + hid_dim*2`. 

`forward` now takes a `context` argument. Inside of `forward`, we concatenate $y_t$ and $z$ as `emb_con` before feeding to the GRU, and we concatenate $y_t$, $s_t$ and $z$ together as `output` before feeding it through the linear layer to receive our predictions, $\hat{y}_{t+1}$.

In [ ]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, dropout):
        super().__init__()

        self.emb_dim = emb_dim  # 256
        self.hid_dim = hid_dim # 512
        self.output_dim = output_dim # 5893
        self.dropout = dropout 
        
        self.embedding = nn.Embedding(output_dim, emb_dim)    # [5893, 256]
        self.rnn = nn.GRU(emb_dim + hid_dim, hid_dim)   # [768, 512]
        self.out = nn.Linear(emb_dim + hid_dim*2, output_dim) # [1280, 5893]
        self.dropout = nn.Dropout(p=dropout, inplace=True)
        
    def forward(self, input, hidden, context): 
        '''
        Inputs: [128]           input/target token, y_t 
                [1, 128, 512]   previous hidden state, s_{t-1}
                [1, 128, 512]   context vector, z 
        '''
        embedded = self.embedding(input).unsqueeze(0) # [1, 128, 256]
        emb_con = self.dropout(torch.cat((embedded, context), dim=-1))  # [1, 128, 768]
        unused_output, hidden = self.rnn(emb_con)  # [1, 128, 512], [1, 128, 512]
        output = torch.cat((emb_con, hidden), dim=2).squeeze()  # [128, 1280]
        prediction = self.out(output)  # Linear Transformation: [128, 1280] --> [128, 5893]
        return prediction, hidden.squeeze()  # [128, 5893], [128, 512]


## Seq2Seq Model

Putting the encoder and decoder together, we get:

![](assets/seq2seq7.png)

Again, in this implementation we need to ensure the hidden dimensions in both the encoder and the decoder are the same.

Briefly going over all of the steps:
- the `outputs` tensor is created to hold all predictions, $\hat{Y}$
- the source sequence, $X$, is fed into the encoder to receive a `context` vector
- the initial decoder hidden state is set to be the `context` vector, $s_0 = z = h_T$
- we use a batch of `<sos>` tokens as the first `input`, $y_1$
- we then decode within a loop:
  - inserting the input token $y_t$, previous hidden state, $s_{t-1}$, and the context vector, $z$, into the decoder
  - receiving a prediction, $\hat{y}_{t+1}$, and a new hidden state, $s_t$
  - we then decide if we are going to teacher force or not, setting the next input as appropriate

In [ ]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self.decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        
    def forward(self, src, trg, teacher_forcing_ratio=0.5):
        batch_size = trg.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim
        # predicted outputs
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        context = self.encoder(src).to(device) # final hidden state returned from encoder is the context vector
        hidden = context # initial decoder hidden state is the context vector
        input = trg[0,:]
        
        for t in range(1, max_len):
            output, unused_hidden = self.decoder(input=input, hidden=hidden, context=context) # [128, 5893], [128, 512]
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            
            top1 = output.argmax(dim=1)  # [1]
            # if tfr = 0.5, then 50% of time the input is set equal to actual target word
            input = trg[t] if teacher_force else top1
            
        return outputs


# Training the Seq2Seq Model


We initialise our encoder, decoder and seq2seq model (placing it on the GPU if we have one). As before, the embedding dimensions and the amount of dropout used can be different between the encoder and the decoder, but the hidden dimensions must remain the same.

In [ ]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, DEC_DROPOUT)

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model = Seq2Seq(enc, dec, device).to(device)

Next, we initialize our parameters. The paper states the parameters are initialized from a normal distribution with a mean of 0 and a standard deviation of 0.01, i.e. $\mathcal{N}(0, 0.01)$. 

It also states we should initialize the recurrent parameters to a special initialization, however to keep things simple we'll also initialize them to $\mathcal{N}(0, 0.01)$.

In [ ]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.normal_(param.data, mean=0, std=0.01)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): GRU(256, 512)
    (dropout): Dropout(p=0.5, inplace=True)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): GRU(768, 512)
    (out): Linear(in_features=1280, out_features=5893, bias=True)
    (dropout): Dropout(p=0.5, inplace=True)
  )
)

We print out the number of parameters.



In [ ]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 14,219,781 trainable parameters


We initiaize our optimizer.

In [ ]:
optimizer = optim.Adam(model.parameters())

We also initialize the loss function, making sure to ignore the loss on `<pad>` tokens.

In [ ]:
PAD_IDX = TRG.vocab.stoi['<pad>']

criterion = nn.CrossEntropyLoss(ignore_index = PAD_IDX)

We then create the training loop...

In [ ]:
def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        src = batch.src.to(device)
        trg = batch.trg.to(device)
        optimizer.zero_grad()
        output = model(src, trg)
        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)
        loss = criterion(output, trg)
        #TODO-Compute the gradients
        loss.backward()
        #TODO- Clip the gradients
        nn.utils.clip_grad_norm_(model.parameters(), clip)
        #TODO update the parameters
        optimizer.step()
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)


...and the evaluation loop, remembering to set the model to `eval` mode and turn off teaching forcing.

In [ ]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src.to(device)
            trg = batch.trg.to(device)

            output = model(src, trg, 0) # turn off teacher forcing
            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

We'll also define the function that calculates how long an epoch takes.

In [ ]:
def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

Then, we train our model, saving the parameters that give us the best validation loss.

In [ ]:
N_EPOCHS = 15
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut2-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

torch.Size([21, 128]) torch.Size([128])
torch.Size([21, 128]) torch.Size([128])
torch.Size([21, 128]) torch.Size([128])
torch.Size([21, 128]) torch.Size([128])
torch.Size([21, 128]) torch.Size([128])
torch.Size([21, 128]) torch.Size([128])
torch.Size([21, 128]) torch.Size([128])
torch.Size([21, 128]) torch.Size([128])
torch.Size([21, 128]) torch.Size([128])
torch.Size([21, 128]) torch.Size([128])
torch.Size([21, 128]) torch.Size([128])
torch.Size([21, 128]) torch.Size([128])
torch.Size([21, 128]) torch.Size([128])
torch.Size([21, 128]) torch.Size([128])
torch.Size([21, 128]) torch.Size([128])
torch.Size([21, 128]) torch.Size([128])
torch.Size([21, 128]) torch.Size([128])
torch.Size([21, 128]) torch.Size([128])
torch.Size([21, 128]) torch.Size([128])
torch.Size([21, 128]) torch.Size([128])
torch.Size([32, 128]) torch.Size([128])
torch.Size([32, 128]) torch.Size([128])
torch.Size([32, 128]) torch.Size([128])
torch.Size([32, 128]) torch.Size([128])
torch.Size([32, 128]) torch.Size([128])


KeyboardInterrupt: 

Finally, we test the model on the test set using these "best" parameters.

In [ ]:
model.load_state_dict(torch.load('tut2-model.pt'))

test_loss = evaluate(model, test_iterator, criterion)

print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

Now generate some translated sentences

In [ ]:
def lookup_words(x, vocab=TRG.vocab):
    if vocab is not None:
        #print("x",x)
        x = [vocab.itos[i] for i in x]

    return [str(t) for t in x]


def generate_translation(model, iterator):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src.to(device)
            trg = batch.trg.to(device)

            output = model(src, trg, 0) # turn off teacher forcing

            _, next_word =output.max(2)
            target = trg.cpu().detach().numpy()
            next_word = next_word.cpu().detach().numpy()
            r, c = next_word.shape
            
            for i in range(c):
                print("Target", lookup_words(target[:,i]))
                print( "Output",lookup_words(next_word[:,i]))
            

In [ ]:
generate_translation(model, test_iterator)